# Widget Example Using bqplot

See https://astrowidgets.readthedocs.io for additional details about the widget, including installation notes.

In [1]:
from astrowidgets.bqplot import ImageWidget

In [2]:
# from ginga.misc.log import get_logger

# logger = get_logger('my viewer', log_stderr=True,
#                     log_file=None, level=30)

In [3]:
w = ImageWidget()

For this example, we use an image from Astropy data repository and load it as `CCDData`. Feel free to modify `filename` to point to your desired image.

Alternately, for local FITS file, you could load it like this instead:
```python
w.load_fits(filename, numhdu=numhdu)
```    
Or if you wish to load a data array natively (without WCS):
```python
from astropy.io import fits
# NOTE: memmap=False is needed for remote data on Windows.
with fits.open(filename, memmap=False) as pf:
    arr = pf[numhdu].data.copy()
w.load_array(arr)
```

In [4]:
filename = 'http://data.astropy.org/photometry/spitzer_example_image.fits'
numhdu = 0

# Loads NDData
# NOTE: Some file also requires unit to be explicitly set in CCDData.
from astropy.nddata import CCDData
ccd = CCDData.read(filename, hdu=numhdu, format='fits')
w.load_nddata(ccd)

Ginga key bindings documented at http://ginga.readthedocs.io/en/latest/quickref.html . Note that not all documented bindings would work here. Please use an alternate binding, if available, if the chosen one is not working.

Here are the ones that worked during testing with Firefox 52.8.0 on RHEL7 64-bit:

Key | Action | Notes
--- | --- | ---
`+` | Zoom in |
`-` | Zoom out |
Number (0-9) | Zoom in to specified level | 0 = 10
Shift + number | Zoom out to specified level | Numpad does not work
` (backtick) | Reset zoom |
Space > `q` > arrow | Pan |
ESC | Exit mode (pan, etc) |
`c` | Center image
Space > `d` > up/down arrow | Cycle through color distributions
Space > `d` > Shift + `d` | Go back to linear color distribution
Space > `s` > Shift + `s` | Set cut level to min/max
Space > `s` > Shift + `a` | Set cut level to 0/255 (for 8bpp RGB images)
Space > `s` > up/down arrow | Cycle through cuts algorithms
Space > `l` | Toggle no/soft/normal lock |

*NOTE: This list is not exhaustive.*

A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

In [5]:
w

ImageWidget(children=(_AstroImage(children=(Figure(axes=[Axis(scale=LinearScale(allow_padding=False, max=1025.…

This next cell captures print outputs. You can pop it out like the viewer above. It is very convenient for debugging purpose.

In [6]:
# Capture print outputs from the widget
display(w.print_out)

AttributeError: 'ImageWidget' object has no attribute 'print_out'

The following cell changes the visibility or position of the cursor info bar.


In [6]:
w.cursor = 'top'  # 'top', 'bottom', None
print(w.cursor)

top


# 👆 ~FAILURE ABOVE~ FIXED! 😃 -- cursor does not move

The rest of the calls demonstrate how the widget API works. Comment/uncomment as needed. Feel free to experiment.

In [7]:
# Programmatically center to (X, Y) on viewer
w.center_on((1, 1))

In [8]:
# Programmatically offset w.r.t. current center
w.offset_by(10, 10)

In [9]:
from astropy.coordinates import SkyCoord

# Change the values here if you are not using given
# example image.
ra_str = '01h13m23.193s'
dec_str = '+00d12m32.19s'
frame = 'galactic'

# Programmatically center to SkyCoord on viewer
w.center_on(SkyCoord(ra_str, dec_str, frame=frame))

# 👆 ~FAILURE ABOVE~ 😕 -- issue is that the test image is in Galactic coordinates so either the frame needs to be galactic here or a different center is needed -- image moves completely out of view

In [10]:
from astropy import units as u

# Change the values if needed.
deg_offset = 0.1 * u.deg

# Programmatically offset (in degrees) w.r.t.
# SkyCoord center
w.offset_by(deg_offset, deg_offset)

In [11]:
# Show zoom level
print(w.zoom_level)

0.4878048780487805


# 👆 ~FAILURE ABOVE~ 😃 fixed! -- zoom_level should never be zero!

In [12]:
w.zoom_level = 1

# 👆 ~FAILURE ABOVE~ -- setting zoom_level to 1 took image out of view -- 😕 now it works, but not sure what I did to fix it

with X: -349501.59 Y: -186964.83


In [13]:
# Programmatically zoom image on viewer
w.zoom(2)

In [14]:
# Capture what viewer is showing and save RGB image.
# Need https://github.com/ejeschke/ginga/pull/688 to work.
w.save('test.png', overwrite=True)

# 👆 FAILURE ABOVE -- saving *downloads* the image but does not put it in the directory notebook is in

In [19]:
# Get all available image stretch options
print(w.stretch_options)

AttributeError: 'ImageWidget' object has no attribute 'stretch_options'

# 👆 FAILURE ABOVE -- There should be stretch_options

In [14]:
# Get image stretch algorithm in use
print(w.stretch)

In [15]:
# Change the stretch
w.stretch = 'histeq'
print(w.stretch)

histeq


# 👆 ~FAILURE ABOVE~ -- changing stretch does not change display -- 😕 the change looks terrible but it does change

In [26]:
# Get all available image cuts options
print(w.autocut_options)

AttributeError: 'ImageWidget' object has no attribute 'autocut_options'

# 👆 FAILURE ABOVE -- There should be autocut options I guess

In [14]:
# Get image cut levels in use
print(w.cuts)

# 👆 FAILURE ABOVE -- this isn't *wrong* but maybe a __str__ would be nice

In [15]:
# Change the cuts by providing explicit low/high values
w.cuts = (10, 15)
print(w.cuts)

(10, 15)


# 👆 ~FAILURE ABOVE~ -- yeah, it is a failure...works now, though 🤷‍♂️ HA HA HA NO -- fails again, WTF? -- AAAH, the issue was with whether the stretch had been set to something.

In [16]:
w.stretch = 'log'

In [24]:
# Change the cuts with an autocut algorithm
w.cuts = 'zscale'
print(w.cuts)

ValueError: Cut levels must be given as (low, high).zscale is not a valid value. cuts must be one of None, an astropy interval, or list/tuple of length 2.

# 👆 FAILURE ABOVE -- yeah, it is a failure...

In [19]:
# This enables click to center.
w.click_center = True

Now, click on the image to center it.

# 👆 ~FAILURE ABOVE~ -- clicking does nothing -- FIXED 😃

Actually, I knew this would be the case....

In [20]:
# Turn it back off so marking (next cell) can be done.
w.click_center = False

In [16]:
# This enables marking mode.
w.start_marking()
print(w.is_marking)

True


# 👆 ~FAILURE ABOVE~ 😃 Fixed -- yeah, it is a failure...

Now, click on the image to mark a point of interest.

In [17]:
# When done, set back to False.
w.stop_marking()
print(w.is_marking)

False


In [21]:
# Get table of markers
markers_table = w.get_all_markers()

# Default display might be hard to read, so we do this
print(f'{"X":^8s} {"Y":^8s} {"Coordinates":^28s}')
for row in markers_table:
    c = row['coord'].to_string('hmsdms')
    print(f'{row["x"]:8.2f} {row["y"]:8.2f} {c}')

   X        Y             Coordinates         
  119.00   242.00 01h13m23.32s +00d12m37.8s
  691.00   209.00 01h12m37.56s +00d11m58.2s
  710.00   305.00 01h12m36.04s +00d13m53.4s
   68.00   291.00 01h13m27.4s +00d13m36.6s


In [55]:
# Erase markers from display
w.remove_all_markers()

The following works even when we have set `w.is_marking=False`. This is because `w.is_marking` only controls the interactive marking and does not affect marking programmatically.

In [44]:
# Programmatically re-mark from table using X, Y.
# To be fancy, first 2 points marked as bigger
# and thicker red circles.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 50,
            'linewidth': 2}
# 👇👇👇👇👇👇 HAD TO ADD DIFFERENT MARKER NAMES FOR THIS TO WORK 👇👇👇👇
w.add_markers(markers_table[:2], marker_name='first')
# You can also change the type of marker to cross or plus
w.marker = {'type': 'cross', 'color': 'cyan', 'radius': 20}
w.add_markers(markers_table[2:], marker_name='second')

/Users/mattcraig/miniconda3/envs/awid-jlab3/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


## 👆 FAILURE -- first two are not showing up as big red circles

In [37]:
# Erase them again
w.remove_all_markers()

# Programmatically re-mark from table using SkyCoord
w.add_markers(markers_table, use_skycoord=True)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Start marking again
w.start_marking()
print(w.is_marking)

In [ ]:
# Stop marking AND clear markers.
# Note that this deletes ALL of the markers
w.stop_marking(clear_markers=True)
print(w.is_marking)

The next cell randomly generates some "stars" to mark. In the real world, you would probably detect real stars using `photutils` package.

In [53]:
import numpy as np
from astropy.table import Table

# Maximum umber of "stars" to generate randomly.
max_stars = 1000

# Number of pixels from edge to avoid.
dpix = 20

# Image from the viewer.
img = w._data #w.viewer.get_image()

# Random "stars" generated.
bad_locs = np.random.randint(
    dpix, high=img.shape[1] - dpix, size=[max_stars, 2])

# Only want those not near the edges.
mask = ((dpix < bad_locs[:, 0]) &
        (bad_locs[:, 0] < img.shape[0] - dpix) &
        (dpix < bad_locs[:, 1]) &
        (bad_locs[:, 1] < img.shape[1] - dpix))
locs = bad_locs[mask]

# Put them in table
t = Table([locs[:, 1], locs[:, 0]], names=('x', 'y'), dtype=('float', 'float'))
print(t)

  x     y  
----- -----
141.0 451.0
941.0 239.0
195.0 321.0
327.0 287.0
108.0 101.0
919.0 352.0
331.0 194.0
533.0 464.0
322.0 298.0
853.0 398.0
  ...   ...
697.0  47.0
437.0 327.0
138.0 393.0
420.0 335.0
744.0 445.0
964.0 480.0
769.0 340.0
149.0 302.0
979.0 399.0
478.0 423.0
789.0 153.0
Length = 482 rows


# 👆 FAILURE ABOVE -- to be fair, this is relying on an implementation detail of ginga

Fixed temporarily by changing `img`

In [56]:
# Mark those "stars" based on given table with X and Y.
w.add_markers(t)

/Users/mattcraig/miniconda3/envs/awid-jlab3/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


# 👆 ~FAILURE ABOVE~ 😃 fixed! -- Really?! Does anything in here work?! This _should_ have been caught by the tests.... 😃 it is now!

The following illustrates how to control number of markers displayed using interactive widget from `ipywidgets`.

In [58]:
# Set the marker properties as you like.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 10,
            'linewidth': 2}

# Define a function to control marker display
def show_circles(n):
    """Show and hide circles."""
    w.remove_all_markers()
    t2show = t[:n]
    w.add_markers(t2show)
    with w.print_out:
        print('Displaying {} markers...'.format(len(t2show)))

We redisplay the image widget below above the slider. Note that the slider affects both this view of the image widget and the one near the top of the notebook.

In [59]:
from IPython.display import display

import ipywidgets as ipyw
from ipywidgets import interactive

# Show the slider widget.
slider = interactive(show_circles,
                     n=ipyw.IntSlider(min=0,max=len(t),step=1,value=0, continuous_update=False))
display(w, slider)

ImageWidget(children=(_AstroImage(children=(Figure(axes=[Axis(scale=LinearScale(allow_padding=False, max=1022.…

interactive(children=(IntSlider(value=0, continuous_update=False, description='n', max=482), Output()), _dom_c…

Now, use the slider. The chosen `n` represents the first `n` "stars" being displayed.